In [118]:
import polars as pl
import os

In [119]:
# load the data as a lazy dataframe
data = pl.scan_csv(r'data\downloaded datasets\retailrocket\events.csv',
                   with_column_names=lambda names: ['Time', 'UserId', 'Type', 'ItemId', 'transactionid'])

print(data.fetch(5))
print(data.collect().shape)

shape: (5, 5)
┌───────────────┬────────┬──────┬────────┬───────────────┐
│ Time          ┆ UserId ┆ Type ┆ ItemId ┆ transactionid │
│ ---           ┆ ---    ┆ ---  ┆ ---    ┆ ---           │
│ i64           ┆ i64    ┆ str  ┆ i64    ┆ str           │
╞═══════════════╪════════╪══════╪════════╪═══════════════╡
│ 1433221332117 ┆ 257597 ┆ view ┆ 355908 ┆ null          │
│ 1433224214164 ┆ 992329 ┆ view ┆ 248676 ┆ null          │
│ 1433221999827 ┆ 111016 ┆ view ┆ 318965 ┆ null          │
│ 1433221955914 ┆ 483717 ┆ view ┆ 253185 ┆ null          │
│ 1433221337106 ┆ 951259 ┆ view ┆ 367447 ┆ null          │
└───────────────┴────────┴──────┴────────┴───────────────┘
(2756101, 5)


In [120]:
# remove transactionid colum
data = data.drop('transactionid')

In [121]:
# check for missing values
data.null_count().collect()

Time,UserId,Type,ItemId
u32,u32,u32,u32
0,0,0,0


Create session ids. each session has a maximum lenght of 30 minutes.

In [122]:
# create a timestamp column 
data = (data
          .with_columns(
              pl.from_epoch("Time", time_unit="ms")
              )
          )

create session ids. Each session has a maximum lenght of 30 minutes.

In [123]:
# sort the dataset on userid and timestamp
data = data.sort(['UserId', 'Time'])

data = (data
       # create two new columns that contain boolean values if the previous row is another user or exceeds the maximum session length
       .with_columns([
           # check if the user_id is different from the previous row
           (pl.col("UserId").diff() != 0).fill_null(True).alias("char_diff"),
           # check if the time difference between the current and previous row exceeds the threshold
           (pl.col("Time").diff().dt.total_minutes().cast(pl.Int64) > 30).fill_null(True).alias("ts_diff"),
       ])

       # create a new column that determines if a new session needs to be started after this row
       .with_columns([
           (pl.col("ts_diff") | pl.col("char_diff")).alias("new_session_mark")
       ])

       # drop the columns that we don't need anymore
       .drop(["char_diff", "ts_diff"])

       # create a new column that contains the session id
       .with_columns([
           (pl.col("new_session_mark").cum_sum()).alias("SessionId")
       ])
       # drop the column that we don't need anymore
       .drop("new_session_mark")
       )
data.collect()

Time,UserId,Type,ItemId,SessionId
datetime[ms],i64,str,i64,u32
2015-09-11 20:49:49.439,0,"""view""",285930,1
2015-09-11 20:52:39.591,0,"""view""",357564,1
2015-09-11 20:55:17.175,0,"""view""",67045,1
2015-08-13 17:46:06.444,1,"""view""",72028,2
2015-08-07 17:51:44.567,2,"""view""",325215,3
…,…,…,…,…
2015-06-10 21:46:08.922,1407575,"""view""",121220,1759829
2015-06-03 15:01:29.991,1407576,"""view""",356208,1759830
2015-05-17 21:48:04.867,1407577,"""view""",427784,1759831


Filter out session ids with only one interaction and also session ids that have items that appear les than 5 times. (item cold start)

In [124]:
data.group_by('SessionId').len().collect()

SessionId,len
u32,u32
1034497,1
176440,1
1582812,2
729605,2
560257,1
…,…
1739551,2
401871,1
984517,1


In [128]:
# Get session ids with more than one interaction in session
SessionIdsWithMoreThanOneInteraction = (data
                 .group_by('SessionId')
                 .agg(pl.col('Type').len().alias('actions'))
                #  .filter(pl.col('actions') > 4)
                #  .select('SessionId')
                 ).collect()

SessionIdsWithMoreThanOneInteraction

SessionId,actions
u32,u32
1193630,1
1559613,1
674421,3
1212774,8
1293908,5
…,…
1521697,1
183237,1
1522611,1


In [84]:
ItemIdsColdstartItems = (data
                         .group_by('ItemId')
                         .agg(pl.col('Type').count().alias('actions'))
                         .filter(pl.col('actions') < 4)
                         .select('ItemId')
                         ).collect()

print(f'ItemIdsColdstartItems: {ItemIdsColdstartItems}')

SessionIdsWithoutColdstartItems = (data
                                   # invert the filter to get the session ids that do not contain coldstart items
                                   .filter(~pl.col('ItemId').is_in(ItemIdsColdstartItems))
                                   .select('SessionId')
                                   ).collect()
print(f'SessionIdsWithoutColdstartItems: {SessionIdsWithoutColdstartItems}')

ItemIdsColdstartItems: shape: (144_113, 1)
┌────────┐
│ ItemId │
│ ---    │
│ i64    │
╞════════╡
│ 241202 │
│ 126761 │
│ 198475 │
│ 61749  │
│ 463507 │
│ …      │
│ 379461 │
│ 66846  │
│ 437402 │
│ 163051 │
│ 296502 │
└────────┘
SessionIdsWithoutColdstartItems: shape: (2_491_203, 1)
┌───────────┐
│ SessionId │
│ ---       │
│ u32       │
╞═══════════╡
│ 1         │
│ 1         │
│ 1         │
│ 2         │
│ 3         │
│ …         │
│ 1759829   │
│ 1759830   │
│ 1759831   │
│ 1759832   │
│ 1759833   │
└───────────┘


In [85]:
# keep the session ids that have more than one interaction and do not contain coldstart items
data = (data
        .filter(pl.col('SessionId').is_in(SessionIdsWithMoreThanOneInteraction))
        .filter(pl.col('SessionId').is_in(SessionIdsWithoutColdstartItems))

        # sort the dataset on userid and timestamp
        .sort(['SessionId', 'Time'])
        ).collect()

display(data)
data.shape

Time,UserId,Type,ItemId,SessionId
datetime[ms],i64,str,i64,u32
2015-09-11 20:49:49.439,0,"""view""",285930,1
2015-09-11 20:52:39.591,0,"""view""",357564,1
2015-09-11 20:55:17.175,0,"""view""",67045,1
2015-08-07 17:51:44.567,2,"""view""",325215,3
2015-08-07 17:53:33.790,2,"""view""",325215,3
…,…,…,…,…
2015-06-17 23:57:31.071,1407527,"""view""",85307,1759774
2015-07-27 05:05:55.003,1407573,"""view""",232069,1759825
2015-07-27 05:07:58.632,1407573,"""view""",116453,1759825


(950047, 5)

Create a train and test set.

In [86]:
min_time = data['Time'].min()
max_time = data['Time'].max()

print(f"First date in the dataset: {min_time}")
print(f"Last date in the dataset: {max_time}")

First date in the dataset: 2015-05-03 03:00:04.384000
Last date in the dataset: 2015-09-18 02:58:58.914000


In [87]:
# take last 7 days of data as test set
test = data.filter(pl.col('Time') > max_time - pl.duration(days=7))

test

Time,UserId,Type,ItemId,SessionId
datetime[ms],i64,str,i64,u32
2015-09-11 20:49:49.439,0,"""view""",285930,1
2015-09-11 20:52:39.591,0,"""view""",357564,1
2015-09-11 20:55:17.175,0,"""view""",67045,1
2015-09-15 17:35:31.705,54,"""view""",388096,66
2015-09-15 17:37:45.550,54,"""view""",283115,66
…,…,…,…,…
2015-09-11 04:39:40.260,1406985,"""view""",425477,1759082
2015-09-11 04:51:24.263,1406985,"""view""",407489,1759082
2015-09-11 04:51:42.207,1406985,"""view""",415976,1759082


In [88]:
# remove test data from the dataset
train = data.filter(pl.col('Time') <= max_time - pl.duration(days=7))

train

Time,UserId,Type,ItemId,SessionId
datetime[ms],i64,str,i64,u32
2015-08-07 17:51:44.567,2,"""view""",325215,3
2015-08-07 17:53:33.790,2,"""view""",325215,3
2015-08-07 17:56:52.664,2,"""view""",259884,3
2015-08-07 18:01:08.920,2,"""view""",216305,3
2015-08-07 18:08:25.669,2,"""view""",342816,3
…,…,…,…,…
2015-06-17 23:57:31.071,1407527,"""view""",85307,1759774
2015-07-27 05:05:55.003,1407573,"""view""",232069,1759825
2015-07-27 05:07:58.632,1407573,"""view""",116453,1759825


Filter out items that are in the test data but not in the train data. 

In [89]:
ItemIdsInTest = test.select('ItemId').unique()
ItemIdsInTrain = train.select('ItemId').unique()

# create a list with the item ids that are in the test set but not in the training set
ItemIdsInTestbutNotInTrain = ItemIdsInTest.filter(~pl.col('ItemId').is_in(ItemIdsInTrain))

# remove items that are in the test set but not in the training set
test = test.filter(~pl.col('ItemId').is_in(ItemIdsInTestbutNotInTrain))

test.shape

(32422, 5)

prepare the data for the model by converting the Time back to unix time

In [90]:
train = train.with_columns(
    pl.col("Time").dt.epoch(time_unit='ms').alias("Time")
)

test = test.with_columns(
    pl.col("Time").dt.epoch(time_unit='ms').alias("Time")
)   

save the datasets

In [91]:
# create directory
os.makedirs('data/processed datasets/retailrocket', exist_ok=True)

train.write_csv('data/processed datasets/retailrocket/events_train.csv',
                separator='\t')
test.write_csv('data/processed datasets/retailrocket/events_test.csv',
               separator='\t')

summary statistics

In [92]:
# concat the train and test data to calculate the statistics
data = pl.concat([train, test])

In [93]:
data.group_by('Type').len()

Type,len
str,u32
"""view""",871411
"""transaction""",20408
"""addtocart""",55056


In [94]:
data.shape

(946875, 5)

In [95]:
def summary_stats(df, dataset_str):
    print(f'Summary statistics for the {dataset_str} dataset: ')
    # calculate the number of unique items
    n_items = df.select('ItemId').n_unique()
    print(f'Number of unique items: {n_items}')
    # calculate the number of sessions
    n_sessions = df.select('SessionId').n_unique()
    print(f'Number of sessions: {n_sessions}')
    # calculate the session length
    session_length = df.group_by('SessionId').agg(pl.count('SessionId').alias('len')).select(pl.col('len').mean())[0, 0]
    print(f'Average session length: {round(session_length, 2)}\n')
    
summary_stats(data, 'whole')
summary_stats(train, 'train')
summary_stats(test, 'test')

Summary statistics for the whole dataset: 
Number of unique items: 93407
Number of sessions: 169852
Average session length: 5.57

Summary statistics for the train dataset: 
Number of unique items: 93407
Number of sessions: 163437
Average session length: 5.6

Summary statistics for the test dataset: 
Number of unique items: 11699
Number of sessions: 6426
Average session length: 5.05

